In [2]:
from ipynb.fs.full.Infer_propagation_path_functions import *

from pymongo import MongoClient

In [3]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [4]:
COLLECTION_NAME = 'data_false_8'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[264361128,
 965856403129745408,
 2371260104,
 1378023550687268872,
 20478016,
 1274557407130083328,
 15987245,
 1021401793266372608,
 1320845430,
 1224685021866483714,
 959953351382364161,
 1189099063054143489,
 1519014456,
 284677980,
 1240653191655698432,
 784866246772727808,
 2263663771,
 1260952232050036736,
 1147021327,
 93930925,
 337286471,
 4831347854,
 1327793345393881089,
 827334883185799168,
 1348293669946851329,
 34922026,
 585777737,
 874807324510113792,
 47598596,
 1427077376,
 3872346202,
 172990190,
 1321831977083932678,
 2670696570,
 26946959,
 20878297,
 396161595,
 1283099384163586049,
 2800470003,
 952050248578486272,
 760149872322510848,
 1275520387460157440,
 2745094971,
 1272473084168192001,
 4926435839,
 858118388,
 882764425693933568,
 41924468,
 717674874,
 252365681,
 3556262056,
 79960355,
 1215397509721313280,
 244164131,
 1350566381654306816,
 23633315,
 20208407,
 3222204881,
 810723502768713728,
 1273638001269248001,
 821922018937290752,
 90877554766843

In [5]:
len(db['followers'].find_one({'user_id': 264361128})['followers'])

462880

In [5]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['user']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    len(cascade_depth_one_user_ids)
    print(cascade_depth_one_user_ids)

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({})
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{264361128}
264361128 Tue Apr 27 17:56:36 +0000 2021
264361128
264361128
[971176632621953024, 857029745464180737, 1375381514, 130713612, 1105774474555785217, 2218639374, 1147021327, 307562513, 35004434, 978692921527586818, 42616853, 42242071, 1252909080, 289703965, 437895198, 1055188190238257152, 1336812699473354754, 774337735897198592, 1314347481963147266, 53231656, 1016749275886206976, 1364929426413277184, 153968684, 1283099384163586049, 166514736, 66070578, 1075194934695731200, 2391949368, 9508922, 1094449606392344576, 912097800661426176, 780530249406697472, 705561147156508673, 1461493826, 20478016, 33110084, 1029780752051920898, 798351157001388033, 1241172871512231936, 1382059887358844930, 950862222661771264, 962156417300938752, 1379909255009042439, 2871410776, 318697561, 3872346202, 2214273115, 30640220, 1012931887227179008, 1228937546970750976, 4758155359, 1379803730128822275, 1356348960852627456, 2278137954, 23533665, 799406180, 948232248867934208, 1247866757643190273, 132941094

In [10]:
users = [971176632621953024, 857029745464180737, 1375381514, 130713612, 1105774474555785217, 2218639374, 1147021327, 307562513, 35004434, 978692921527586818, 42616853, 42242071, 1252909080, 289703965, 437895198, 1055188190238257152, 1336812699473354754, 774337735897198592, 1314347481963147266, 53231656, 1016749275886206976, 1364929426413277184, 153968684, 1283099384163586049, 166514736, 66070578, 1075194934695731200, 2391949368, 9508922, 1094449606392344576, 912097800661426176, 780530249406697472, 705561147156508673, 1461493826, 20478016, 33110084, 1029780752051920898, 798351157001388033, 1241172871512231936, 1382059887358844930, 950862222661771264, 962156417300938752, 1379909255009042439, 2871410776, 318697561, 3872346202, 2214273115, 30640220, 1012931887227179008, 1228937546970750976, 4758155359, 1379803730128822275, 1356348960852627456, 2278137954, 23533665, 799406180, 948232248867934208, 1247866757643190273, 1329410944125071360, 216770668, 1017620237460688896, 920137534947991553, 3227363439, 791305575925776384, 1220106319186755585, 758496146436202502, 253374586, 418410620, 2670696570, 1372542611366494215, 1014717562003714048, 247355521, 39258242, 607869060, 703568996851777536, 1273621781840973830, 334176405, 803576199716872192, 1641713816, 1286967450, 18055323, 301836444, 153516187, 347025569, 604960930, 1173940122141757440, 1101587859772006401, 1160339624, 3556262056, 1237540282935037952, 1194962793767804928, 1397418156, 1327423161282007040, 227461294, 1368219445051686915, 1250910351484489730, 15395003, 548030657, 1255413993612677121, 22137027, 1197263042674855937, 1148673429123608576, 549009612, 1334680023752577024, 115529940, 1347012150343172097, 4886962393, 633151710, 1357378367574581248, 1343731237236723713, 1061099584070524929, 1133574998952632320, 331825387, 130924781, 127381742, 1259653809036541953, 586176749, 803075022105391104, 1265643958945886209, 260483317, 2806794486, 1351682944176250881, 1229782292723380225, 959953351382364161, 1093554427, 488196349, 37007615, 1128350164786216962, 1341958885385535489, 1427077376, 1078432341188665345, 280840452, 1061205204580864001, 1164634067846991872, 1374302138797486081, 878101341326987268, 85379343, 1376747471582289922, 1378854141011709952, 1224685021866483714, 250272023, 1836024090, 2399987996, 1313604732892700679, 743946528, 1348410889536466947, 79960355, 1147261854894481409, 1266154256471437312, 965750090215419904, 707224118186348545, 1242118936080068608, 80402739, 1110788857119694859, 1880926518, 1377033435559124994, 331663677, 267768124, 854941337933807616, 874807324510113792, 925736706669727744, 853837446676713472, 2349465926, 1296531627066236928, 2705531210, 1218228814968446977, 841259058715906049, 63277398, 773291662638362624, 472478040, 1191893329270771712, 50479452, 1902991712, 827334883185799168, 1182727824924389382, 1386770809646157824, 869648505761013760, 1358152721677828098, 1360787156067897344, 977044405591842816, 1091846620059189248, 1019617501632782336, 74471792, 28262769, 858455496042332161, 1359947150948401152, 43508093, 1386995154884698113, 1316015083273609216, 542751106, 460370313, 794679690, 245891470, 1349053243054305280, 3326716301, 15554960, 1214341075076308998, 834468240, 1261598102814892032, 940325211987202051, 2322557343, 1304267993371860993, 846717230641561602, 1277976484225757186, 2877295024, 1330994940575248384, 1111964082, 4689732019, 988488283268272128, 1039004244240621568, 1338687138280349696, 2321502648, 2365966777, 1384206911373860874, 338784704, 1360400311186317313, 1324208800241340416, 1380195926644506626, 795022664767250432, 1361007264652533765, 279210452, 1225156030222323713, 1105784251197415427, 1591960026, 2446143962, 285778397, 3299299804, 961792232939556864, 1029683683, 1196678718980263936, 1324925760692396032, 68293108, 739318262, 1192768890608275456, 1293682087740899328, 1318478275581267969, 30106118, 142500359, 1327793345393881089, 1350491668596797440, 529533450, 1175933203028348929, 1052185369112924163, 1357849592398766083, 237406736, 512721426, 983905717588344833, 1332389333559492608, 771730778539438080, 1331426186602409984, 273766945, 480096802, 1110514212, 702316442474012672, 1307896646362161158, 15987245, 1262271337671917568, 21365300, 1313059988433260544, 95107639, 914315513815339008, 1009607984811405314, 1318588346122252289, 3905231419, 1378023550687268872, 1244410923361521664, 585777737, 787345993, 1893429836, 1032680185643065345, 478855762, 582203986, 714213250674597888, 1288340054, 1354938253997031427, 2207887956, 397632089, 291521114, 4159261277, 486251102, 803951014143348736, 1038934241684406273, 956699572771831808, 1324081601278521344, 372621928, 3782339177, 1326931555286212608, 197071472, 410286704, 850412011771097089, 3168590448, 877952592223096833, 958916234459754496, 1334936008895434756, 1326619320173146113, 36969087, 344908417, 1367921390125424641, 1296074901586092034, 782339095225769984, 977392461243928576, 3183397512, 1380038014357889032, 749775975479709696, 965856403129745408, 1240989403188690945, 926603843001028610, 291549851, 532554400, 1170413987646754816, 1124705089925599233, 1895836326, 392221352, 382145192, 2269293227, 704353017106206721, 387969710, 2695373490, 837174038359322624, 1365915984394031105, 1346195160892370946, 1260952232050036736, 1368677976544514053, 1004827598604722176, 1477958341, 1537243854, 199252687, 297777874, 1365643373000982529, 1254139704775237641, 974995710503456769, 1090994195706318848, 288520930, 1044673978927927296, 780622566, 885818088, 79626985, 872864297587757056, 1253554802551926787, 185637611, 1380240535378554885, 2354217709, 806717688705875968, 3302578926, 3188871920, 940435908054073344, 2727305971, 762389795037294592, 3900689141, 20677368, 1235650395260612608, 829723901227790336, 409441021, 77028093, 3353090813, 1423123200, 1265583508749238274, 2513435397, 1320378785519276034, 1311068128298897409, 390384393, 2333911817, 1227786738183090177, 2152381201, 592853778, 529771285, 1353029920952258560, 39686936, 97569565, 1317783993631244288, 2802580255, 1210748744234225664, 77767467, 1201706377829175296, 1383851475021881351, 1618731829, 1285933389418303489, 309513014, 966089528, 1312374452118679552, 1133857742622404608, 2745094971, 2906833726, 629723968, 1377474357145010177, 22594372, 4903488322, 1199461189, 456383304, 958560441352925184, 1334606608249344003, 1351547892838903808, 2195313484, 1314168911982743552, 1315681839051284480, 17365843, 30448472, 1344084438574116865, 2186683228, 1114255734730977280, 770693533443432450, 1377764690474962947, 23757668, 1120467812, 2863983468, 401615732, 2220890995, 344501110, 1343257659982671873, 2244842360, 2200316797, 183866238, 1104637277102399488, 1352357248866136064, 3106323325, 89719683, 985696510846304258, 1275527781556731904, 434271114, 875824534858944512, 1374189171317698563, 299434896, 717867921, 1056816019, 709019843664609280, 1023849606327754752, 2737439639, 125752225, 3224705958, 761831637600571392, 2540712878, 528837552, 531907506, 1869812658, 924629743932145664, 229317558, 2905983926, 1277045217376399361, 1345553590161731584, 247663545, 622142396, 45417405, 1386165150093172738, 1275879755934359553, 1255512405280542720, 3883733953, 309021637, 19153863, 1300495634814959616, 1069004459894657025, 1188270685883392001, 1269436634619154433, 1112802254, 827776024586887168, 1104553888361918465, 1274564780028641281, 803154902, 1959467989, 950072516118114304, 1378121192025042947, 331686877, 966279343684116480, 26399716, 1246179828199538688, 990565367209656320, 1348293669946851329, 49273832, 1084318527299842048, 866616301, 4916628460, 828306414, 25814008, 18527224, 30315515, 949909874988765184, 720740855557058565, 365714435, 47598596, 1343148028467806209, 3235648520, 22643723, 941549209337053185, 4707089419, 1701071887, 4480818194, 2614703132, 702368798, 1888322592, 1333450396782694402, 265819175, 1351900200785354758, 75205678, 1380918527394205699, 476982320, 1154493867758149653, 594955315, 779072275299573760, 1344687454847176704, 160930875, 1273619423098617856, 2321134655, 3445142593, 2573833283, 1380263256376377351, 330488901, 66348103, 2864303175, 1350281792478183426, 485260360, 448568395, 1371612332418289665, 1315638398007169024, 1033811583137443840, 1352410467302563843, 286467155, 965650667330207745, 1241144300890710019, 3927594072, 1965321306, 1225632411184070656, 927743921618239489, 758828157986148352, 394206307, 310314084, 1713718376, 1381024210806509570, 2669110378, 1166099564, 935334985430700032, 1364601904773349380, 1236495619365068801, 2963696751, 3439197291, 1313571391027978241, 295965813, 1123615378926657537, 1324540136365809664, 1194661428600422401, 3382944893, 1358172011004977152, 877562301049704449, 1352023562060771329, 1359231961110204419, 804220046, 3885526161, 824860521912422406, 2332146833, 1270862028484292614, 868950962223947777, 1190584502302593024, 1361479387745558534, 2322994327, 2263663771, 538504348, 1190993523157487618, 573781149, 1291721968740433925, 1263068742247948288, 1289896822270496770, 1278576272193585152, 320566450, 2335790261, 1320124622138990596, 811195897963024384, 1006165705543299074, 941505630807916547, 1214356044257775616, 963579441082376192, 50521292, 703413002691698688, 23485648, 2907364560, 24368338, 813814505771769857, 1060723513584013312, 1925197015, 146322653, 924920653836271616, 977195853239803905, 954563820, 844584174, 1260271725578727429, 374797552, 36582643, 22039797, 1350972677314502661, 1247926462805614592, 832636522935390209, 866708194512556032, 25462022, 148090117, 1005149876345278474, 49755395, 2366166278, 41866505, 44633356, 1094223554244165632, 453565705, 1095894811238780929, 2162154768, 1260746801159774209, 949734555346440192, 254854424, 292085017, 1389993241, 310646043, 1860881694, 709157030062215168, 763606304, 720416030418460672, 1181590793917734913, 361332010, 2858382638, 3080820021, 2923396405, 1266789798649319424, 1005249848, 1116867902, 602434879, 225449279, 1323141975365677062, 894407224147156992, 1772248387, 780594585575620609, 1362807989468491776, 2494860618, 1025983599311343616, 2200681805, 1198989562552496128, 2596152654, 3221697872, 949009007171301378, 2163246427, 1043399899818209280, 1238233368296189952, 1001161074031329280, 473902436, 3069982054, 1124508707092664322, 872927259681796096, 1352076842728087552, 1322293277685669888, 17911160, 807779014861606913, 1291696075795988481, 1055160333344620546, 1353479840913977344, 1331735934, 21048705, 267857282, 1121913906274938880, 824482823759396866, 1249281349946691584, 1902212492, 2276693388, 2381036940, 1281848883044159494, 155757967, 1276905502224588802, 450393495, 1446165912, 1377955028875538434, 1253490292877074443, 2200307099, 486882714, 1202099152160804864, 1085511349097152512, 1217272426859507712, 892890102602629126, 93930925, 956701381812944897, 3317894574, 409427378, 1303584117922181121, 2975784370, 50927030, 247903672, 897895142262222849, 485090758, 92792262, 2997659079, 347581897, 962427696406921216, 1955091913, 302804428, 247991761, 1291471961877512193, 1337101691968217092, 1213921217234649088, 1368969933053386760, 22281685, 2515480024, 964481629841514496, 1008141037599150081, 997977152678187009, 810343300372774912, 981669197133615104, 461780457, 1360877586901999617, 1237120961763311616, 1278669318025248769, 28921330, 822525941347799041, 2188885496, 1385071284518756353, 1110281349284458496, 1343228276903436291, 227534342, 712794709358280704, 1273638001269248001, 719638088956252160, 97054224, 1248441604068818949, 256644626, 1088594610841153536, 1281660066756800512, 1321875799302090755, 793688003352952832, 876151390116118528, 20803098, 2776530459, 1017711323730227200, 479200798, 1375171350801883137, 1362170729584877569, 446723616, 462552610, 244164131, 524758567, 479905321, 1072207543730982913, 971162424228368385, 34922026, 32667181, 943894548727652352, 775137093773365249, 910900402979209218, 1154283898358001664, 1278383581627387909, 702798054357909504, 1358172955876880384, 288874039, 1357834309411954689, 2749783609, 1346751245755822082, 3083609669, 1091717345091964928, 861642313, 1177400099648880640, 938196565, 1291450266227744768, 295581274, 1611619933, 456767070, 491783777, 98887266, 598613606, 1345062483485024257, 1274557407130083328, 1267177478122061828, 978859148707356672, 209677939, 923418229, 1079446900145831936, 20874877, 1235661125389148160, 1713702529, 903432699418886144, 4049196673, 1177281477622542336, 1325504451189338112, 64577167, 1357728949875400704, 372110997, 141956759, 768531472093413376, 774961418202415104, 34471581, 511352479, 813467298, 541748899, 23025314, 138196647, 749958546004054017, 212739753, 1307956303646449665, 1208098653732143109, 959872921111027712, 23027368, 746546862, 1967007409, 1349676207906971648, 1125349446265581568, 1262467333651673092, 1321256031251537921, 1182071056888143872, 746906352014921728, 2993841853, 716326749823438848, 1328895462930341889, 31854279, 1354953886558277636, 535643852, 785318920056102913, 1357394836735864833, 983023320, 1284264132611584001, 19650266, 705595570807824384, 1197842528353042433, 80762594, 2889709282, 19945186, 1366168968889794564, 2293772007, 1277095356468166656, 1259727472494592000, 172990190, 52958962, 1379934875260698634, 169504501, 826550703258791937, 2309334781, 1144089676384485376, 133773057, 1240653191655698432, 157568770, 2388694785, 1375734797922758659, 1375932720082083840, 1587425034, 2851428106, 381220620, 766895643218575360, 1272473084168192001, 721367070923890689, 562372370, 1350133804351037440, 312110870, 1032621235874791425, 825332512532475904, 1347701690410037249, 310963993, 1042802725304512518, 1276281841314664448, 3011327783, 100640553, 734130260627820545, 1286348984945057792, 1244000199539716097, 957933668558815237, 866348764432392192, 837717490453086208, 1164839793173225474, 259790657, 1053705566441431040, 811619500093276162, 888229083049586688, 52598603, 3436543817, 1355838185457250306, 41224014, 798517731129692160, 107593551, 1361111463742627840, 1266228011537326080, 69211993, 827588850079907841, 284677980, 202385245, 2863007584, 2192672610, 784866246772727808, 1269852033219264512, 167702378, 1346448425739460608, 106768242, 25374581, 903617947616378881, 2920011642, 1326507724809641984, 974118649962336257, 190265219, 1241942110984712192, 23482251, 1251551861716520961, 1350868550093684737, 567463830, 463308702, 327659422, 76599202, 4647645089, 142823331, 633319333, 1314355997687906304, 1941467048, 1039847541980979200, 1264587723249238016, 825847389525311493, 1117886491047469056, 1192248667547275264, 979871402747748352, 951618360021729280, 382250935, 40411062, 1205970580777906178, 713385914, 1326265953685008384, 1236686555944812544, 1322923458351624193, 1325610670763728897, 1378884215769554944, 794326664826077184, 1307556703295229952, 1319207131195887616, 1328517792652537858, 1350679542139035656, 401100756, 1245465104356986881, 1360847857994010625, 53546967, 1004817203609423873, 1053876188, 1288974989899845636, 716784760354443264, 966506468, 795527387639222273, 860825203802333184, 997943099786711040, 17012720, 23287792, 1050222679394009093, 883048436, 705824874854424576, 1239373893636218880]

In [11]:
len(users)

935

In [26]:
def get_followers_len(user_id, db):
    user = db['followers'].find_one({'user_id' : user_id})
    if user == None:
        return 0
    followers = user['followers']
    return len(followers)

In [28]:
for user in users:
    print(user, get_followers_len(user, db))

971176632621953024 1607
857029745464180737 2
1375381514 836
130713612 30
1105774474555785217 2
2218639374 121
1147021327 145
307562513 192
35004434 75
978692921527586818 10
42616853 1033
42242071 133
1252909080 21
289703965 208
437895198 5749
1055188190238257152 459
1336812699473354754 71
774337735897198592 42
1314347481963147266 5
53231656 842
1016749275886206976 1891
1364929426413277184 38
153968684 158
1283099384163586049 331
166514736 39
66070578 254
1075194934695731200 4976
2391949368 343
9508922 193219
1094449606392344576 5227
912097800661426176 322
780530249406697472 3461
705561147156508673 289
1461493826 2450
20478016 934
33110084 271
1029780752051920898 32
798351157001388033 2417
1241172871512231936 44
1382059887358844930 62
950862222661771264 48
962156417300938752 79
1379909255009042439 12
2871410776 833
318697561 100
3872346202 60912
2214273115 716
30640220 225
1012931887227179008 10
1228937546970750976 367
4758155359 8
1379803730128822275 16
1356348960852627456 39
227813795

In [6]:
add_depth_nodes(root, 2, tweet_user_ids, db, COLLECTION_NAME)

[Node('/264361128/971176632621953024', created_at='Tue Apr 27 20:17:56 +0000 2021', difference_seconds=8480.0), Node('/264361128/857029745464180737', created_at='Wed Apr 28 16:03:18 +0000 2021', difference_seconds=79602.0), Node('/264361128/1375381514', created_at='Mon May 03 09:31:24 +0000 2021', difference_seconds=488088.0), Node('/264361128/130713612', created_at='Fri Apr 30 15:13:40 +0000 2021', difference_seconds=249424.0), Node('/264361128/1105774474555785217', created_at='Wed Apr 28 01:39:19 +0000 2021', difference_seconds=27763.0)]
 user_id: 264361128, diff_s: 0
├──  user_id: 971176632621953024, diff_s: 8480.0
│   ├──  user_id: 1110281349284458496, diff_s: 90713.0
│   ├──  user_id: 79053827, diff_s: 11574.0
│   ├──  user_id: 981669197133615104, diff_s: 8015.0
│   ├──  user_id: 907956112619577345, diff_s: 1101.0
│   ├──  user_id: 951618360021729280, diff_s: 7033.0
│   ├──  user_id: 2591181143, diff_s: 32305.0
│   ├──  user_id: 3145379514, diff_s: 14987.0
│   └──  user_id: 343675

Node('/264361128', created_at='Tue Apr 27 17:56:36 +0000 2021', difference_seconds=0)

In [7]:
remove_duplicates(root)

1058
keep the olderst node Node('/264361128/704353017106206721/920065753', created_at='Tue Apr 27 21:13:30 +0000 2021', difference_seconds=5007.0)
keep the olderst node Node('/264361128/265819175/989966720', created_at='Thu May 06 02:17:11 +0000 2021', difference_seconds=1992.0)
keep the olderst node Node('/264361128/702798054357909504/155190873', created_at='Sat May 01 17:30:16 +0000 2021', difference_seconds=95098.0)
3179


In [8]:
add_depth_nodes(root, 3, tweet_user_ids, db, COLLECTION_NAME)

[Node('/264361128/705561147156508673/2446143962', created_at='Wed Apr 28 15:25:29 +0000 2021', difference_seconds=1585.0), Node('/264361128/318697561/987859981021188096', created_at='Wed Apr 28 01:11:53 +0000 2021', difference_seconds=23472.0), Node('/264361128/3872346202/912865137169637376', created_at='Tue Apr 27 18:31:56 +0000 2021', difference_seconds=1855.0), Node('/264361128/3872346202/3131604482', created_at='Wed Apr 28 01:14:57 +0000 2021', difference_seconds=26036.0), Node('/264361128/3872346202/950135295101386752', created_at='Wed Apr 28 00:38:16 +0000 2021', difference_seconds=23835.0)]
 user_id: 264361128, diff_s: 0
├──  user_id: 857029745464180737, diff_s: 79602.0
├──  user_id: 1105774474555785217, diff_s: 27763.0
├──  user_id: 2218639374, diff_s: 78693.0
├──  user_id: 1147021327, diff_s: 189.0
├──  user_id: 307562513, diff_s: 60226.0
├──  user_id: 35004434, diff_s: 86100.0
├──  user_id: 978692921527586818, diff_s: 20994.0
├──  user_id: 42242071, diff_s: 696.0
├──  user_id

Node('/264361128', created_at='Tue Apr 27 17:56:36 +0000 2021', difference_seconds=0)

In [9]:
remove_duplicates(root)

290
keep the olderst node Node('/264361128/762389795037294592/535643852/155190873', created_at='Wed Apr 28 22:38:21 +0000 2021', difference_seconds=100825.0)
keep the olderst node Node('/264361128/146322653/813850311978811393/920065753', created_at='Tue Apr 27 21:13:30 +0000 2021', difference_seconds=11127.0)
513


In [12]:
add_depth_nodes(root, 4, tweet_user_ids, db, COLLECTION_NAME)

[Node('/264361128/3872346202/3033060855/1691804784', created_at='Tue Apr 27 22:09:51 +0000 2021', difference_seconds=10350.0), Node('/264361128/3872346202/825518317/22643723', created_at='Tue Apr 27 19:47:34 +0000 2021', difference_seconds=5078.0), Node('/264361128/3872346202/825518317/2402535456', created_at='Tue Apr 27 20:18:29 +0000 2021', difference_seconds=6933.0), Node('/264361128/3872346202/825518317/1281960265588846592', created_at='Tue Apr 27 18:36:53 +0000 2021', difference_seconds=837.0), Node('/264361128/3872346202/825518317/1541659320', created_at='Tue Apr 27 21:59:17 +0000 2021', difference_seconds=12981.0)]
 user_id: 264361128, diff_s: 0
├──  user_id: 857029745464180737, diff_s: 79602.0
├──  user_id: 1105774474555785217, diff_s: 27763.0
├──  user_id: 2218639374, diff_s: 78693.0
├──  user_id: 1147021327, diff_s: 189.0
├──  user_id: 307562513, diff_s: 60226.0
├──  user_id: 35004434, diff_s: 86100.0
├──  user_id: 978692921527586818, diff_s: 20994.0
├──  user_id: 42242071, d

Node('/264361128', created_at='Tue Apr 27 17:56:36 +0000 2021', difference_seconds=0)

In [15]:
# import tree
from anytree.importer import DictImporter
from anytree import RenderTree
importer = DictImporter()
with open('data_false_2_tree.json', 'r') as fp:
    data = fp.readline()
data = json.loads(data)
root = importer.import_(data)
print(RenderTree(root))

AnyNode(created_at='Wed May 12 06:53:36 +0000 2021', difference_seconds=0, name=57441414)
├── AnyNode(created_at='Wed May 12 12:27:05 +0000 2021', difference_seconds=20009.0, name=1373852964809494539)
├── AnyNode(created_at='Wed May 12 12:42:44 +0000 2021', difference_seconds=20948.0, name=538542132)
├── AnyNode(created_at='Wed May 12 21:14:26 +0000 2021', difference_seconds=51650.0, name=3059007541)
├── AnyNode(created_at='Wed May 12 07:25:24 +0000 2021', difference_seconds=1908.0, name=2298634316)
├── AnyNode(created_at='Wed May 12 16:19:34 +0000 2021', difference_seconds=33958.0, name=726161631160455168)
│   └── AnyNode(created_at='Wed May 12 17:02:49 +0000 2021', difference_seconds=2595.0, name=1104841482085773313)
├── AnyNode(created_at='Wed May 12 10:58:08 +0000 2021', difference_seconds=14672.0, name=1122425638399422464)
├── AnyNode(created_at='Wed May 12 12:43:30 +0000 2021', difference_seconds=20994.0, name=2588803166)
├── AnyNode(created_at='Wed May 12 06:58:48 +0000 2021', d

In [13]:
add_depth_nodes(root, 10, tweet_user_ids, db, COLLECTION_NAME)

[AnyNode(created_at='Thu May 13 03:54:38 +0000 2021', difference_seconds=26120.0, name=240318436), AnyNode(created_at='Wed May 12 08:42:07 +0000 2021', difference_seconds=84.0, name=14432483), AnyNode(created_at='Wed May 12 10:37:57 +0000 2021', difference_seconds=73.0, name=397672800), AnyNode(created_at='Wed May 12 11:55:54 +0000 2021', difference_seconds=8093.0, name=822109063550877696), AnyNode(created_at='Wed May 12 16:55:51 +0000 2021', difference_seconds=26090.0, name=950799352133574657)]
 user_id: 57441414, diff_s: 0
├──  user_id: 1373852964809494539, diff_s: 20009.0
├──  user_id: 538542132, diff_s: 20948.0
├──  user_id: 3059007541, diff_s: 51650.0
├──  user_id: 2298634316, diff_s: 1908.0
├──  user_id: 726161631160455168, diff_s: 33958.0
│   └──  user_id: 1104841482085773313, diff_s: 2595.0
├──  user_id: 1122425638399422464, diff_s: 14672.0
├──  user_id: 2588803166, diff_s: 20994.0
├──  user_id: 19456096, diff_s: 312.0
├──  user_id: 526794855, diff_s: 391.0
│   ├──  user_id: 86

AnyNode(created_at='Wed May 12 06:53:36 +0000 2021', difference_seconds=0, name=57441414)

In [14]:
remove_duplicates(root)

2
keep the olderst node AnyNode(created_at='Wed May 12 22:45:39 +0000 2021', difference_seconds=17632.0, name=719638088956252160)
2


In [83]:
# export tree
from anytree.exporter import JsonExporter, DictExporter
exporter = DictExporter()
data = exporter.export(root)
with open(COLLECTION_NAME + '_tree.json', 'w') as fp:
    json.dump(data, fp)    

In [18]:
from anytree import Node, RenderTree, AsciiStyle, LevelOrderGroupIter
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter
l =10
# all_names = [node.name for node in PreOrderIter(root, maxlevel=l)]
# level_names = [[node.name for node in children] for children in LevelOrderGroupIter(root)][l]
all_names = [node.name for node in PreOrderIter(root, maxlevel=l)]
level_names  =  [[node.name for node in children] for children in LevelOrderGroupIter(root)][l]
print(len(all_names), len(level_names))
for thing in all_names:
    if thing in level_names: 
        level_names.remove(thing)
print(len(level_names), level_names)


IndexError: list index out of range

In [12]:

%store level_names

Stored 'level_names' (list)


In [7]:
for user in users:
    print(user, get_cascade_size(user, COLLECTION_NAME, db))

264361128 3895


In [8]:
# Check if the text is same
db[COLLECTION_NAME].distinct('retweet_count')

[0, 4570]

In [9]:
db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')

[4570]

In [13]:
db['data_false_7'].distinct('retweeted_status.quoted_status.retweet_count')

[2753]

In [10]:
# Check if the text is same
cache = db[COLLECTION_NAME].distinct('retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweet_count':x})['text'])
    print('*'*5)

[0, 4570]
RT @Liz_Wheeler: Joe Rogan is correct about the COVID vaccine for kids &amp; young healthy people.

https://t.co/AxGwFxAMVB
*****
Joe Rogan is correct about the COVID vaccine for kids &amp; young healthy people.

https://t.co/AxGwFxAMVB
*****


In [11]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.retweet_count':x})['id'])
    print('*'*5)

[4570]
1387103453000830977
*****


In [12]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.quoted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.quoted_status.retweet_count':x})['retweeted_status.quoted_status.id'])
    print('*'*5)

[]


In [13]:
db['timeline_extended'].count()

C:\Users\Akila\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


616